In [215]:
import numpy as np
import scipy
import pandas as pnd
import theano
import theano.tensor as T
from lasagne.updates import adam as optimize
import time
from collections import Counter
from theano.tensor.extra_ops import to_one_hot

In [216]:
table_csv = pnd.read_csv('data/train.csv')

In [217]:
table = np.int32(table_csv.as_matrix())
#table = table[(table[:, 0] < 10000) * (table[:, 1] < 1000)]
users_n = len(np.unique(table[:, 0]))
films_n = len(np.unique(table[:, 1]))
#users_n = 10000
#films_n = 1000

valid_size = int(0.1 * len(table))
valid = table[-valid_size:]
table = table[:-valid_size]

mu = table[:, 2].mean()

print(mu)
print(table.shape)

3.60411121861
(89164901, 3)


In [218]:
f = 40
U = theano.shared(np.float32(np.random.normal(0, 0.01, (users_n, f))))
F = theano.shared(np.float32(np.random.normal(0, 0.01, (films_n, f))))
bu = theano.shared(np.float32(np.random.normal(0, 0.01, users_n)))
bf = theano.shared(np.float32(np.random.normal(0, 0.01, films_n)))


In [219]:
lmbd = 0.02
lrt = 0.005
user_batch, film_batch, ratings = T.ivector(), T.ivector(), T.ivector()

predict = mu + bu[user_batch] + bf[film_batch] + T.batched_dot(U[user_batch], F[film_batch])
loss = ((predict-ratings)**2).mean() + lmbd*((U[user_batch]**2).mean() + (F[film_batch]**2).mean() +\
                               (bu[user_batch]**2).mean() + (bf[film_batch]**2).mean())
    
def train_loop(u, f, r, U, F, bu, bf):
    prediction = mu + bu[u] + bf[f] + T.dot(U[u], F[f])
    err = prediction - r
    lss = (err**2) + lmbd*((U[u]**2).mean() + (F[f]**2).mean() +\
                               (bu[u]**2).mean() + (bf[f]**2).mean())
    dbu = np.float32(2.) * err + (lmbd * np.float32(2.) * bu[u])
    dbf = np.float32(2.) * err + (lmbd * np.float32(2.) * bf[f])
    dU  = (np.float32(2.) * err * F[f]) + lmbd * np.float32(2.) * U[u]
    dF  = (np.float32(2.) * err * U[u]) + lmbd * np.float32(2.) * F[f]
    U -= lrt * dU
    F -= lrt * dF
    bu -= lrt * dbu
    bf -= lrt * dbf
    return lss, prediction

result, updates = theano.scan(train_loop, sequences=[user_batch, film_batch, ratings], non_sequences=[U, F, bu, bf])
updates = optimize(loss, [U, F, bu, bf], lrt)

In [220]:
predict_fn = theano.function([user_batch, film_batch], predict)
train_fn = theano.function([user_batch, film_batch, ratings], [loss, predict], updates=updates)

In [221]:
start = time.time()
batch_size = 1024*128
#idxs = np.arange(0, len(table), dtype=np.int32)
#np.random.shuffle(idxs)  
#table = table[idxs]
print(time.time() - start)
start = time.time()
train_fn(table[:batch_size, 0], table[:batch_size, 1], table[:batch_size, 2])
print(time.time() - start)

0.00016832351684570312
0.8761703968048096


In [ ]:
epochs = 100
best_valid = 100.
best_model = None#(U.get_value(), F.get_value(), bu.get_value(), bf.get_value())
#U.set_value(best_model[0])
#F.set_value(best_model[1])
#bu.set_value(best_model[2])
#bf.set_value(best_model[3])

for epoch in range(epochs):
    idxs = np.arange(0, len(table), dtype=np.int32)
    np.random.shuffle(idxs)
    table = table[idxs]
    res = 0
    cnt = 0
    start = time.time()
    train_rmse = 0
    for i in range(0, len(table), batch_size):
        if i + batch_size > len(table):
            break
        idx = np.arange(i, i+batch_size)
        tmp = train_fn(table[idx, 0], table[idx, 1], table[idx, 2])
        res += tmp[0]
        train_rmse += ((table[idx, 2] - tmp[1])**2).sum()
        cnt += 1
        
    print('epoch', epoch)
    print('loss ', res/cnt)
    print('train rmse:', (train_rmse/(cnt*batch_size))**0.5)
    
    valid_pred = predict_fn(valid[:, 0], valid[:, 1])
    valid_true = valid[:, 2]
    valid_scor = ((valid_pred-valid_true)**2).mean()**0.5
    if valid_scor < best_valid:
        best_valid = valid_scor
        best_model = (U.get_value(), F.get_value(), bu.get_value(), bf.get_value())
    print('valid rmse:', valid_scor)
    print('time:', time.time()-start)

epoch 0
loss  0.792711975472
train rmse: 0.888616551899
valid rmse: 0.866574399877
time: 459.4908375740051
epoch 1
loss  0.651829953903
train rmse: 0.804264242284
valid rmse: 0.855397510438
time: 423.01605319976807
epoch 2
loss  0.599090366785
train rmse: 0.770317314823
valid rmse: 0.858595190681
time: 424.7209253311157


KeyboardInterrupt: 

In [223]:
test_table = pnd.read_csv('data/test-ids.csv')
test = np.int32(test_table.as_matrix())

In [224]:
best_valid

0.8553975104379512

In [226]:
U.set_value(best_model[0])
F.set_value(best_model[1])
bu.set_value(best_model[2])
bf.set_value(best_model[3])

predictions = []
for i in range(0, len(test), batch_size):
    predictions += list(predict_fn(test[:, 1], test[:, 2]))
    print(min((i+batch_size), len(test))/len(test))
    
#predictions = np.array(predictions)
with open('s.txt', 'w+') as out:
    out.write('Id,Prediction\n')
    for i, p in zip(list(test[:, 0]), predictions):
        out.write(str(i) + ',' + str(p) + '\n')

0.0930648007128682
0.1861296014257364
0.2791944021386046
0.3722592028514728
0.46532400356434095
0.5583888042772092
0.6514536049900773
0.7445184057029456
0.8375832064158137
0.9306480071286819
1.0
